# Stable Video Diffusion XT 1.1 on Amazon SageMaker

> [Original Code](https://github.com/garystafford/svdxt-sagemaker-huggingface)

Stability AI의 [Stable Video Diffusion XT(SVT-XT) 1.1](https://medium.com/r/?url=https%3A%2F%2Fstability.ai%2Fstable-video) 기반 모델은 정지 이미지를 conditioning frame으로 사용하여 4초 분량의 짧은 비디오를 생성하는 diffusion 모델입니다.

노트북은 Amazon SageMaker의 SVT-XT 기반 모델로 지원되는 [Asynchronous Inference Endpoint](https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference.html)를 구성, 생성 및 호출하는 과정을 포함합니다.

![Architecture](assets/architecture.png)

## 1: Install Required Packages


In [ ]:
%%sh

sudo apt-get update -qq -y && sudo apt-get upgrade -qq -y
sudo apt-get install git libgl1 ffmpeg git-lfs wget -y

In [ ]:
%pip install sagemaker boto3 botocore ffmpeg-python ipython diffusers pywget imageio-ffmpeg -Uq

In [ ]:
# restart kernel 1x when installing new packages

import os

os._exit(00)

## 2: Prepare the SVD-XT Model for Inference

모델을 준비하기 위해 아래와 같은 단계를 수행합니다.

1. Hugging Face에서 모델 아티팩트 다운로드
2. 사용자 정의 infernece 스크립트 추가
3. 모델 아티팩트 압축
4. 모델 압축파일을 S3에 업로드

### 2.1: Import Packages and Set SageMaker Variables

In [1]:
import os
import json
import sagemaker
import boto3
from botocore.exceptions import ClientError
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.async_inference.async_inference_config import AsyncInferenceConfig
from sagemaker.s3 import s3_path_join

/home/yoo/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/yoo/.config/sagemaker/config.yaml


In [ ]:
sm_session_bucket = None

sm_session = sagemaker.Session()

if sm_session_bucket is None and sm_session is not None:
    # set to default bucket if a bucket name is not given
    sm_session_bucket = sm_session.default_bucket()
try:
    sm_role = sagemaker.get_execution_role()
except ValueError:
    iam_client = boto3.client("iam")
    sm_role = iam_client.get_role(RoleName="sagemaker_execution_role")["Role"]["Arn"]

In [ ]:
# name of packaged model archive file
MODEL_ARCHIVE = "model_svdxt.tar.gz"

In [ ]:
print(f"sagemaker role arn: {sm_role}")
print(f"sagemaker bucket: {sm_session.default_bucket()}")
print(f"sagemaker session region: {sm_session.boto_region_name}")

### 2.2: Download the Model Artifacts from Hugging Face

- Hugging Face에서 모델 아티팩트를 다운로드하는 데 6-7분이 걸립니다. 
- `/home/sagemaker-user`에 마운트된 `/dev/nvme1n1` 볼륨을 확인하여 충분한 공간이 있는지 확인합니다. 약 34GB의 공간이 필요합니다.

In [ ]:
%%sh

df -h /home/sagemaker-user

In [ ]:
%%sh

git lfs install

In [ ]:
%%time
%%sh

rm -rf stable-video-diffusion-img2vid-xt
git clone --depth 1 https://huggingface.co/stabilityai/stable-video-diffusion-img2vid-xt

### 2.3: Add Custom Inference Script

In [ ]:
import shutil

destination = "stable-video-diffusion-img2vid-xt"

shutil.copy("code/inference.py", destination)
shutil.copy("code/requirements.txt", destination)

### 2.4: TAR GZIP Model Artifacts

- 최종 모델 아카이브 파일은 14-15GB가 되며, 패키징 및 압축하는 데 35-40분이 걸릴 수 있습니다.
- 터미널에서 15초마다 모델 아카이브 파일 file의 크기를 지속적으로 폴링할 수 있습니다.
    ```sh
    while sleep 15; do ls -la model_v2.tar.gz; done
    ```


In [ ]:
%%time

import os
import tarfile

TAR_MODE = "w:gz"


def create_tar_archive(folder_path, output_tar_file):
    """
    Create a tar archive from a folder, excluding hidden files.

    :param folder_path: The path to the folder to be archived.
    :param output_tar_file: The path to the output tar file.
    """
    with tarfile.open(output_tar_file, TAR_MODE) as tar:
        for root, dirs, files in os.walk(folder_path):
            files = [f for f in files if not f[0] == "."]
            dirs[:] = [d for d in dirs if not d[0] == "."]
            for file in files:
                file_path = os.path.join(root, file)
                tar.add(file_path, arcname=os.path.relpath(file_path, folder_path))
                print(f"Added {file_path} to the archive.")


create_tar_archive(destination, MODEL_ARCHIVE)

### 2.2-2.4: Alternate Method if Model Already Exists in S3

- 모델 아카이브 파일이 S3에 이미 있는 경우 위의 1-3단계를 건너뜁니다. Amazon S3 사전 서명 URL을 만들고 URL을 사용하여 모델 패키지를 다운로드합니다.
- 모델 아티팩트와 TAR GZIP을 다운로드하는 단계를 대체합니다.
- 이 단계는 동일 AWS 리전에서 4-7분이 걸립니다.


In [ ]:
# %%time

# import os
# from pywget import wget

# presigned_s3_url = "<YOUR_PRESIGNED_URL_GOES_HERE>"

# wget.download(presigned_s3_url, MODEL_ARCHIVE)

### 2.5: Copy Model Artifacts to S3

- 동일 AWS 리전에서 모델 아카이브 파일을 Amazon S3에 복사하는 데 2~3분이 걸리며, 이 파일의 크기는 약 14GB입니다.

In [ ]:
%%time

s3_client = boto3.client("s3")

response = s3_client.upload_file(
    MODEL_ARCHIVE,
    sm_session_bucket,
    f"async_inference/model/{MODEL_ARCHIVE}",
)

## 3: Deploy Model to Amazon SageMaker Endpoint


### 3.1: Deploy Model to Amazon SageMaker Endpoint

- Amazon SageMaker Asynchronous Inference Endpoint를 배포하는 데는 5~7분이 걸립니다.

In [ ]:
env = {
    "SAGEMAKER_MODEL_SERVER_TIMEOUT": "3600",
    "TS_MAX_RESPONSE_SIZE": "1000000000",
    "TS_MAX_REQUEST_SIZE": "1000000000",
    "MMS_MAX_RESPONSE_SIZE": "1000000000",
    "MMS_MAX_REQUEST_SIZE": "1000000000",
}

huggingface_model = HuggingFaceModel(
    model_data=s3_path_join(
        "s3://", sm_session_bucket, f"async_inference/model/{MODEL_ARCHIVE}"
    ),
    transformers_version="4.37.0",
    pytorch_version="2.1.0",
    py_version="py310",
    env=env,
    role=sm_role,
)

In [ ]:
# where the response payload or error will be stored
async_config = AsyncInferenceConfig(
    output_path=s3_path_join("s3://", sm_session_bucket, "async_inference/output"),
    failure_path=s3_path_join(
        "s3://", sm_session_bucket, "async_inference/output_errors"
    ),
)

In [ ]:
%%time

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.4xlarge", # or ml.g5.2xlarge
    async_inference_config=async_config,
)

In [ ]:
endpoint_name = predictor.endpoint_name

In [3]:
with open("endpoint.txt", "w") as f:
    json.dump({
        "endpoint_name": endpoint_name
    }, f)